# Demo using Pangeo to run ASP

For starters, we'll just issue system commands from the Jupyter notebook, following the skysat example in the ASP handbook. Note you can also open a terminal if you prefer (File-->New--Terminal)

In [ ]:
import os
os.environ["PATH"] += ':/home/jovyan/StereoPipeline/bin'

In [ ]:
# Analysis packages
import gcsfs
import numpy as np
import rasterio
import rasterio.plot
import geopandas as gpd
from shapely import wkt
# Visualization packages
import matplotlib.pyplot as plt
%matplotlib inline
#%load_ext wurlitzer

In [ ]:
import subprocess
import sys

def run_bash_command(cmd):
    """Call a system command through the subprocess python module."""
    print(cmd)
    try:
        retcode = subprocess.call(cmd, shell=True)
        if retcode < 0:
            print("Child was terminated by signal", -retcode, file=sys.stderr)
        else:
            print("Child returned", retcode, file=sys.stderr)
    except OSError as e:
        print("Execution failed:", e, file=sys.stderr)

## Get a reference DEM

we'll use SRTM from NASA. This requires credentials in a .netrc file:

NOTE: could create a function here to get SRTM, TDX, or NED based on frame footprints

## Create a DEM with 2 skysat video frames

* Download frames and metadata locally for processing

In [ ]:
# Load geopandas GeoDataFrame to keep track of frames
# Load frames GeoDataFrame
fs = gcsfs.GCSFileSystem(project='pangeo-181919')

def fix_polygon_wkt(string):
    '''returns shapely geometry from reformatted WKT'''
    pre = string[:-2]
    first_point = string.split(',')[0].split('(')[-1]
    fixed = f'{pre},{first_point}))'
    return wkt.loads(fixed)

with fs.open('pangeo-data/skysat/breckenridge/video/frame_index.csv') as f:
    tmp = gpd.pd.read_csv(f)

# NOTE: timestamps not best for index since precision to second has repeat values
#tmp = tmp.set_index(gpd.pd.to_datetime(tmp['datetime']))
tmp['datetime'] = gpd.pd.to_datetime(tmp.datetime)
tmp['geom'] = tmp.geom.apply(fix_polygon_wkt)
tmp['path'] = '/pangeo-data/skysat/breckenridge/video/frames/' + tmp['name'] + '.tiff'

gf = gpd.GeoDataFrame(tmp, crs = {'init': 'epsg:4326'}, geometry='geom')

In [ ]:
gf.head()

In [ ]:
path = 'pangeo-data/skysat/breckenridge/video'
fs.get(f'{path}/frame_index.csv', 'frame_index.csv')
fs.get(f'{path}/ref_dem.tif', 'ref_dem.tif')

v1 = '1225648254.44006968_sc00004_c1_PAN'
v2 = '1225648269.40892076_sc00004_c1_PAN'

fs.get(f'{path}/frames/{v1}.tiff', f'{v1}.tiff')
fs.get(f'{path}/frames/{v2}.tiff', f'{v2}.tiff')


In [ ]:
def cam_gen(frame, refDEM, frameIndex): 
    """ Settings hard-coded for SkySat cameras """
    cmd = f'''cam_gen {frame}.tiff  \
  --reference-dem {refDEM} --focal-length 553846.153846            \
  --optical-center 1280 540 --pixel-pitch 1 --height-above-datum 4000 \
  --refine-camera --frame-index {frameIndex}          \
  --gcp-std 1e-2 -o {frame}.tsai --gcp-file {frame}.gcp 2>&1 | tee cam_gen.log'''
    run_bash_command(cmd)

In [ ]:
refDEM = 'ref_dem.tif'
frameIndex= 'frame_index.csv'
cam_gen(v1, refDEM, frameIndex)

In [ ]:
cam_gen(v2, refDEM, frameIndex)

In [ ]:
def map_project(image, camera, output, refDEM, lon0, lat0):
    cmd = f'''mapproject --t_srs \
  '+proj=stere +lat_0={lat0} +lon_0={lon0} +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m' \
  {refDEM} {image} {camera} {output} 2>&1 | tee mapproject.log'''
    run_bash_command(cmd)

In [ ]:
# NOTE: get approximate lon,lat center of frame, could alternatively use reference dem
with rasterio.open(refDEM) as src:
    lon,lat0 = src.lnglat()
    lon0 = 360 + lon

#test = gf.query('name == @v1')
#lon0 = 360 + test.centroid.x.values[0]
#lat0 = test.centroid.y.values[0]
print(lon0, lat0)

In [ ]:
image = v1+'.tiff'
camera = v1+'.tsai'
output = v1+'_map.tiff'
map_project(image, camera, output, refDEM, lon0, lat0)

In [ ]:
image = v2+'.tiff'
camera = v2+'.tsai'
output = v2+'_map.tiff'
map_project(image, camera, output, refDEM, lon0, lat0)

In [ ]:
def bundle_adjust(images, cameras, gcps, outdir):
    '''for now pass inputs as lists '''
    images = ' '.join(images)
    cameras = ' '.join(cameras)
    gcps = ' '.join(gcps)
    cmd = f'''bundle_adjust {images} {cameras} {gcps} -o {outdir} -t nadirpinhole \
 --datum WGS84 --ip-per-tile 2000 --robust-threshold 100 --max-iterations 500   \
 --overlap-limit 4 --inline-adjustments --camera-weight 0                       \
 --disable-tri-ip-filter --disable-pinhole-gcp-init --skip-rough-homography     \
 --force-reuse-match-files 2>&1 | tee bundle_adjust.log'''
    run_bash_command(cmd)

In [ ]:
images = [x+'.tiff' for x in (v1,v2)]
cameras = [x+'.tsai' for x in (v1,v2)]
gcps = [x+'.gcp' for x in (v1,v2)]

In [ ]:
bundle_adjust(images, cameras, gcps, outdir='ba/run')

In [ ]:
def run_diagnostics(baDir,refDEM,lon0,lat0):
    cmd = f'''pc_align --max-displacement 1000 --csv-format 1:lon,2:lat,3:height_above_datum   \
   ref_dem.tif {baDir}/run-final_residuals_no_loss_function_pointmap_point_log.csv     \
   --save-transformed-source-points -o {baDir}/run 2>&1 | tee diagnose_pc_align.log &&

point2dem --stereographic --proj-lon {lon0} --proj-lat {lat0}            \
   --tr 30 --csv-format 1:lon,2:lat,3:height_above_datum {baDir}/run-trans_source.csv 2>&1 | tee diagnose_point2dem.log &&

geodiff --absolute --csv-format 1:lon,2:lat,3:height_above_datum \
  ref_dem.tif {baDir}/run-trans_source.csv -o {baDir}/run 2>&1 | tee diagnose_geodiff.log'''
    run_bash_command(cmd)

In [ ]:
run_diagnostics('ba', refDEM, lon0, lat0)

In [ ]:
# Do a refined bundle ajust and check errors
def bundle_adjust_refine(images, cameras, baDir, outdir):
    images_str = ' '.join(images)
    cameras_str = ' '.join(cameras)
    cmd = f'''bundle_adjust {images_str}  {cameras_str} -o {outdir}               \
   -t nadirpinhole --max-iterations 0 --overlap-limit 1 --inline-adjustments \
   --camera-weight 0 --initial-transform {baDir}/run-transform.txt 2>&1 | tee bundle_adjust_refine.log'''
    run_bash_command(cmd)

In [ ]:
baDir = 'ba'
outdir = 'ba_align/run'
images = [x+'.tiff' for x in (v1,v2)]
cameras = ['ba/run-'+x+'.tsai' for x in (v1,v2)]

bundle_adjust_refine(images, cameras, baDir, outdir)

In [ ]:
run_diagnostics('ba_align', refDEM, lon0, lat0)

#### NOTE that differences have been reduced

In [ ]:
image = f'{v1}.tiff'
camera = f'ba_align/run-run-{v1}.tsai'
output = f'{v1}_map_align.tif'
map_project(image, camera, output, refDEM, lon0, lat0)

In [ ]:
image = f'{v2}.tiff'
camera = f'ba_align/run-run-{v2}.tsai'
output = f'{v2}_map_align.tif'
map_project(image, camera, output, refDEM, lon0, lat0)

In [ ]:
def stereo_dem(image1, image2, camera1, camera2, output, lon0, lat0):
    cmd = f'''stereo {image1} {image2} {camera1} {camera2} {output} --session-type nadirpinhole \
--cost-mode 4 --stereo-algorithm 2 --corr-seed-mode 1 \
--alignment-method affineepipolar --corr-tile-size 9000 \
--num-matches-from-disp-triplets 10000 --unalign-disparity 2>&1 | tee stereo_dem.log'''
    run_bash_command(cmd)

In [ ]:
# NOTE: input original unprojected images, but with camera models based on aligned
image1 = f'{v1}.tiff'
image2 = f'{v2}.tiff'
camera1 = f'ba_align/run-run-{v1}.tsai'
camera2 = f'ba_align/run-run-{v2}.tsai'
output = 'stereo_align/run'

In [ ]:
stereo_dem(image1, image2, camera1, camera2, output, lon0, lat0)

In [ ]:
def point2dem(stereoDir, lon0, lat0):
    cmd = f'point2dem --stereographic --proj-lon {lon0} --proj-lat {lat0} --tr 4  \
    --errorimage {stereoDir}/run-PC.tif 2>&1 | tee point2dem.log'
    run_bash_command(cmd)

In [ ]:
stereoDir = 'stereo_align'
point2dem(stereoDir, lon0, lat0)

# Examine quality of resulting DEM

In [ ]:
!gdalinfo -stats stereo_align/run-IntersectionErr.tif | grep Mean

In [ ]:
# NOTE: second DEM is subsampled to match first (so 30m --> 4m)
!geodiff --absolute stereo_align/run-DEM.tif {refDEM} -o abs
!gdalinfo -stats abs-diff.tif | grep Mean

In [ ]:
# NOTE: don't take absolute value of difference
!geodiff stereo_align/run-DEM.tif {refDEM} -o dif
!gdalinfo -stats dif-diff.tif | grep Mean

In [ ]:
# Hillshade matching illumination from metadata
!gdaldem hillshade -az 163 -alt 32 stereo_align/run-DEM.tif stereo_align/run-hillshade.tif 

In [ ]:
# Plot original frames
fig, axes = plt.subplots(2,1,figsize=(11,8.5), sharex=True, sharey=True)

gsPath = f'gs://pangeo-data/skysat/breckenridge/video/frames/{v1}.tiff'
with rasterio.open(gsPath) as src:
    #print(src.profile)
    rasterio.plot.show(src, ax=axes[0], cmap='gray', title=v1)
    
gsPath = f'gs://pangeo-data/skysat/breckenridge/video/frames/{v2}.tiff'
with rasterio.open(gsPath) as src:
    #print(src.profile)
    rasterio.plot.show(src, ax=axes[1], cmap='gray', title=v2)

In [ ]:
# Plot map-projected frames
fig, axes = plt.subplots(3,1,figsize=(17,11), sharex=True, sharey=True)

frame = v1 + '_map_align.tif'
with rasterio.open(frame) as src:
    #print(src.profile)
    rasterio.plot.show(src, ax=axes[0], cmap='gray', title=v1)
    
frame = v2 + '_map_align.tif'
with rasterio.open(frame) as src:
    #print(src.profile)
    rasterio.plot.show(src, ax=axes[1], cmap='gray', title=v2)
    
with rasterio.open('stereo_align/run-hillshade.tif') as src:
    #print(src.profile)
    rasterio.plot.show(src, ax=axes[2],  cmap='gray', title='Hillshade')

In [ ]:
# Some basic quantitative figures
with rasterio.open('ref_dem.tif') as src:
    print(src.profile)
    data = src.read()
    #data[data==src.nodata] = np.nan

plt.title('Reference DEM')
plt.imshow(data[0])
plt.colorbar(shrink=0.6);

In [ ]:
# Stereo DEM
with rasterio.open('stereo_align/run-DEM.tif') as src:
    print(src.profile)
    data = src.read()
    data[data==src.nodata] = np.nan

plt.title('Stereo DEM')
plt.imshow(data[0])
plt.colorbar(shrink=0.6);

In [ ]:
# Difference between Stereo and Reference from 'geodiff command'
with rasterio.open('abs-diff.tif') as src:
    print(src.profile)
    data = src.read()
    data[data==src.nodata] = np.nan

plt.title('abs(Stereo DEM - Ref DEM)')
plt.imshow(data[0], cmap='plasma')
cb = plt.colorbar(shrink=0.6)
cb.set_label('[meters]')

In [ ]:
# Histogram of differences
results = plt.hist(data[np.isfinite(data)], bins=50)

In [ ]:
with rasterio.open('dif-diff.tif') as src:
    print(src.profile)
    data = src.read()
    data[data==src.nodata] = np.nan

plt.figure()
plt.title('Stereo DEM - Ref DEM')
plt.imshow(data[0], cmap='bwr', vmin=-30, vmax=30)
cb = plt.colorbar(shrink=0.6)
cb.set_label('[meters]')

# Histogram of differences
plt.figure()
results = plt.hist(data[np.isfinite(data)], bins=50)
plt.axvline(0, color='red', linestyle='dashed')
plt.xlabel('Elevation difference [m]')
plt.ylabel('#pixels')